In [ ]:
from commonfunctions import *
from skimage import io
from skimage.color import rgba2rgb, rgb2gray
from skimage import filters
from skimage.feature import hog
import numpy as np
import cv2
import sys
import os
from PHOG import *
from Faces import *
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn import svm
import pickle

In [ ]:
imgs = []
labels = []

In [ ]:
def ReadFile(path,label,imgs,labels):
    files = os.listdir(path)
    num_images = len(files)
    count = 0
    for f in files:
        img = io.imread(os.path.join(path, f), as_gray=False)
        if len(img.shape) != 2:
            img = rgb2gray(img)
        # img = cv2.resize(img, dsize=(60, 60), interpolation=cv2.INTER_CUBIC)
        imgs.append(img)
        labels.append(label)
        count += 1
        if count>=100:
            return

In [ ]:
ReadFile("./train/happy","happy",imgs,labels)
ReadFile("./train/sad","sad",imgs,labels)
ReadFile("./train/neutral","neutral",imgs,labels)
# ReadFile("./train/fearful","fear",imgs,labels)
# ReadFile("./train/surprised","surprise",imgs,labels)
ReadFile("./train/happy","happy",imgs,labels)
ReadFile("./train/sad","sad",imgs,labels)
# ReadFile("./train/fearful","fear",imgs,labels)
ReadFile("./train/neutral","neutral",imgs,labels)
# ReadFile("./train/surprised","surprise",imgs,labels)
ReadFile("./train/happy","happy",imgs,labels)
ReadFile("./train/sad","sad",imgs,labels)
ReadFile("./train/neutral","neutral",imgs,labels)

In [ ]:
print(len(imgs))
print(len(labels))

In [ ]:
def LPQ(img, window = 3): 
    # img = fftpack.fft2(img)
    # img_ft = np.fft.fftshift(img_ft)
    # img = np.divide(img, np.sum(img))
    lpq_hist = np.zeros((0))
    patch_height = patch_width = window
    w = img.shape[0]
    h = img.shape[1]
    block1 = img[0:(w//2), 0:h//2]
    block2 = img[(w//2):w, 0:h//2]
    block3 = img[0:(w//2), (h//2):h]
    block4 = img[(w//2):w, (h//2):h]
    block_list = [block1,block2,block3,block4]
    mid = window // 2
    count = 0
    for block in block_list:
        block_w, block_h= block.shape
        block_hist = np.zeros(256)
        for i in range (patch_width , block_w-patch_width):
            for j in range (patch_height, block_h - patch_width):
                temp = 0
                patch = block[i - mid : i + mid + 1 , j - mid : j + mid + 1]
                patch = np.fft.fft2(patch)
                patch = np.fft.fftshift(patch)
                if(np.imag(patch[mid-1][mid]) > 0):
                    temp += 1
                if(np.real(patch[mid-1][mid])> 0):
                    temp += 2
                if(np.imag(patch[mid][0])> 0):
                    temp += 4
                if(np.real(patch[mid][0])> 0):
                    temp += 8
                if(np.imag(patch[mid][mid-1])> 0):
                    temp += 16
                if(np.real(patch[mid][mid-1])> 0):
                    temp += 32
                if(np.imag(patch[mid-2][mid-1])> 0):
                    temp += 64
                if(np.real(patch[mid-2][mid-1])> 0):
                    temp += 128
                # lpq_hist.append(temp)
                block_hist[temp] += 1
        block_hist = block_hist.astype('uint8')
        lpq_hist = np.append(lpq_hist,block_hist)
    return lpq_hist

In [ ]:
cnt=  0
features_3 = []
for img in imgs:
    phog = PHOG(img, 40, 3)
    lpq = LPQ(img,3)
    temp = np.concatenate((phog, lpq))
    features_3.append(temp)
    cnt += 1
    if(cnt % 100 == 0):
        print(cnt)
features_3 = np.array(features_3)

In [ ]:
cnt=  0
features_phog = []
for img in imgs:
    phog = PHOG(img, 40, 3)
    features_phog.append(phog)
    cnt += 1
    if(cnt % 100 == 0):
        print(cnt)
features_phog = np.array(features_phog)

In [ ]:
labels = np.array(labels)
labels[labels=="happy"] = 1
labels[labels=="sad"] = 2
labels[labels=="angry"] = 3
labels[labels=="fear"] = 4
labels[labels=="surprise"] = 3
labels[labels=="neutral"] = 5
labels = labels.astype('int32')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features_3, labels, test_size=0.2,stratify=labels)

# pca = PCA(n_components=110)
# x_train = pca.fit_transform(x_train)
clf3 = svm.SVC()
model = clf3.fit(x_train, y_train)

# new = pca.transform(features_2[0].reshape((1, -1)))
# x_test = pca.fit_transform(x_test)
pred = clf3.predict(x_test)
acc = (np.sum(y_test == pred) / len(y_test))*100
print(f'Accuracy when using PHOG + LPQ : {acc} %')
print(x_test.shape)
# target_names = ['happy', 'sad', 'angry', 'fear', 'surprise']
# print(classification_report(y_test, pred, target_names=target_names))

In [ ]:
# x_test = []
# y_test = []
# faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# def getCroppedFaces(faceimg):
#     # colorB = np.copy(faceimg[:,:,0])
#     # colorR = np.copy(faceimg[:,:,2])
#     # faceimg[:,:,0] = colorR
#     # faceimg[:,:,2] = colorB
#     faces = faceCascade.detectMultiScale(faceimg,minNeighbors=5)
#     Cropped_faces = []
#     for face in faces:
#         Cropped_faces.append(np.copy(faceimg[face[1]:face[1]+face[3],face[0]:face[0]+face[2],:]))
#     return Cropped_faces[0]

# for i in range(1,2):
#     tempimg = io.imread('Images/'+str(i)+'.jpg')
#     tempimg = getCroppedFaces(tempimg)
#     tempimg = rgb2gray(tempimg)
#     tempimg = cv2.resize(tempimg, dsize=(60, 60), interpolation=cv2.INTER_CUBIC)
#     phog = PHOG(tempimg, 40, 3)
#     lpq = LPQ(tempimg,3)
#     temp = np.concatenate((phog, lpq))
#     x_test.append(temp)

# # show_images(x_test)
# y_test = ["happy", "sad", "happy","sad", "happy", "happy","happy"]
# x_test = np.array(x_test)


In [ ]:
# print(clf3.predict(x_test))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features_phog, labels, test_size=0.2,stratify=labels)

# pca = PCA(n_components=150)
# features_new = pca.fit(x_train)
clf = svm.SVC(kernel='rbf')
model = clf.fit(x_train, y_train)

# new = pca.transform(features_2[0].reshape((1, -1)))
pred = clf.predict(x_test)
acc = (np.sum(y_test == pred) / len(y_test))*100
print(f'Accuracy when using only PHOG: {acc} %')